In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

In [3]:
print(findFiles('data/names/*.txt'))

['data/names/Czech.txt', 'data/names/German.txt', 'data/names/Arabic.txt', 'data/names/Japanese.txt', 'data/names/Chinese.txt', 'data/names/Vietnamese.txt', 'data/names/Russian.txt', 'data/names/French.txt', 'data/names/Irish.txt', 'data/names/English.txt', 'data/names/Spanish.txt', 'data/names/Greek.txt', 'data/names/Italian.txt', 'data/names/Portuguese.txt', 'data/names/Scottish.txt', 'data/names/Dutch.txt', 'data/names/Korean.txt', 'data/names/Polish.txt']


In [4]:
import unicodedata
import string

In [5]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

In [9]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD',s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [10]:
print(unicodeToAscii('Ślusàrski'))

Slusarski


In [18]:
# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

In [19]:
# Read a file and split it into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

In [16]:
filename = findFiles('data/names/*.txt')[0]
os.path.splitext(os.path.basename(filename))[0]

'Czech'

In [23]:
category_lines['Greek']

['Adamidis',
 'Adamou',
 'Agelakos',
 'Akrivopoulos',
 'Alexandropoulos',
 'Anetakis',
 'Angelopoulos',
 'Antimisiaris',
 'Antipas',
 'Antonakos',
 'Antoniadis',
 'Antonopoulos',
 'Antonopoulos',
 'Antonopoulos',
 'Arvanitoyannis',
 'Avgerinos',
 'Banos',
 'Batsakis',
 'Bekyros',
 'Belesis',
 'Bertsimas',
 'Bilias',
 'Blades',
 'Bouloukos',
 'Brisimitzakis',
 'Bursinos',
 'Calogerakis',
 'Calpis',
 'Chellos',
 'Christakos',
 'Christodoulou',
 'Christou',
 'Chrysanthopoulos',
 'Chrysanthopoulos',
 'Comino',
 'Close',
 'Close',
 'Close',
 'Close',
 'Close',
 'Close',
 'Close',
 'Close',
 'Dalianis',
 'Danas',
 'Dasios',
 'Demakis',
 'Demarchis',
 'Demas',
 'Demetrious',
 'Dertilis',
 'Diakogeorgiou',
 'Dioletis',
 'Dounias',
 'Dritsas',
 'Drivakis',
 'Eatros',
 'Egonidis',
 'Eliopoulos',
 'Forakis',
 'Fotopoulos',
 'Fourakis',
 'Frangopoulos',
 'Galanopoulos',
 'Garofalis',
 'Gavril',
 'Gavrilopoulos',
 'Georgeakopoulos',
 'Geracimos',
 'Gianakopulos',
 'Giannakopoulos',
 'Giannakos',
 '

Making these letters "one hot"

In [24]:
import torch

In [46]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x 1_n_letters> tensor
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(lineToTensor('Jones').size())

torch.Size([5, 1, 57])


In [29]:
enumerate('Jones')

Define the model now!

In [68]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
        
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [69]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [76]:
input = lineToTensor('asdfjkl')
hidden = torch.zeros(1, n_hidden)
output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.9734, -2.8987, -2.8916, -2.8084, -2.8256, -2.8982, -2.9237, -2.8870,
         -2.9557, -2.8994, -2.8292, -2.9081, -2.9521, -2.9233, -2.8917, -2.8412,
         -2.8692, -2.8678]], grad_fn=<LogSoftmaxBackward>)


Make functions that help with the training process

In [77]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('Japanese', 3)


In [96]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype = torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Korean / line = Ryu
category = Italian / line = Marmo
category = German / line = Maas
category = Russian / line = Billevich
category = Greek / line = Missiakos
category = Irish / line = Muirchertach
category = Italian / line = Bellandini
category = Polish / line = Skala
category = Dutch / line = Kappel
category = Korean / line = Choe


Actually do the training!

In [205]:
criterion = nn.NLLLoss()

In [206]:
learning_rate = 0.005

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    
    rnn.zero_grad()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    
    loss = criterion(output, category_tensor)
    loss.backward()
    
    # Add parameters' gradients o their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
    
    return output, loss.item()

In [209]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

In [210]:
for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss
    
    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))
    
    # Add current loss avt to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 12s) 2.0010 Ansaldi / Italian ✓
10000 10% (0m 23s) 3.5328 Bugg / Chinese ✗ (English)
15000 15% (0m 34s) 1.5817 Kattan / Arabic ✓
20000 20% (0m 45s) 2.9153 Szewc / English ✗ (Polish)
25000 25% (0m 57s) 0.9708 Courtemanche / French ✓
30000 30% (1m 8s) 5.1989 Boulos / Greek ✗ (Arabic)
35000 35% (1m 20s) 1.9804 Proulx / Spanish ✗ (French)
40000 40% (1m 34s) 0.8112 Paterson / Scottish ✓
45000 45% (1m 46s) 2.8270 Koning / Chinese ✗ (Dutch)
50000 50% (1m 57s) 0.1630 O'Grady / Irish ✓
55000 55% (2m 9s) 0.8956 Touma / Arabic ✓
60000 60% (2m 20s) 3.1551 Whalen / German ✗ (Irish)
65000 65% (2m 31s) 1.5633 Pinho / Portuguese ✓
70000 70% (2m 43s) 1.7669 Tsen / Dutch ✗ (Chinese)
75000 75% (2m 54s) 2.3344 Mcnab / Scottish ✗ (Irish)
80000 80% (3m 6s) 2.5090 Santos / Arabic ✗ (Spanish)
85000 85% (3m 17s) 0.4414 Castro / Portuguese ✓
90000 90% (3m 29s) 0.6736 Kouches / Greek ✓
95000 95% (3m 40s) 0.4477 Naoimhin / Irish ✓
100000 100% (3m 52s) 1.1087 Meeuwsen / Dutch ✓
